In [2]:
from scipy import stats
import numpy as np

In [8]:
# Hypothesis testing with one sample

# If the population standard deviation is somehow known, p-values can be
# found using a normal distribution

h = 15  # hypothesis, mu is <= 15

n = 10
mu = 15.1  # Observed average
sigma = 0.5  # Population standard deviation

pval = stats.norm.cdf(h, mu, (sigma / np.sqrt(n))) # Right tail test
pval_lefttail1 = 1 - stats.norm.cdf(h, mu, (sigma / np.sqrt(n))) # Left tail test
pval_lefttail2 = stats.norm.sf(h, mu, (sigma / np.sqrt(n))) # Left tail test

print(pval)
print(pval_lefttail1)
print(pval_lefttail2)



0.2635446284327698
0.7364553715672302
0.7364553715672302


In [9]:
# another known sigma
h = 16.43

n = 15
mu = 16
sigma = 0.8

pval = stats.norm.sf(h, mu, (sigma / np.sqrt(n)))

print(pval)

0.018683635713606015


In [41]:
# proportion, two-tailed

h = 0.50  # 50 percent

n = 100
p = 0.53   # proportion observed

# works, but there's probably a better way
if h < p:
    right_tail = (stats.norm.cdf(h, p, np.sqrt(h * (1 - h) / n)))
    left_tail = (stats.norm.sf(h, h + (h - p), np.sqrt(h * (1 - h) / n)))
else:
    right_tail = (stats.norm.cdf(h, h + (h - p), np.sqrt(h * (1 - h) / n)))
    left_tail = (stats.norm.sf(h, p, np.sqrt(h * (1 - h) / n)))
    
print(right_tail, left_tail)
pval = right_tail + left_tail
print(pval)




0.27425311775007344 0.27425311775007344
0.5485062355001469


In [42]:
# another proportion
h = 0.30  # 30 percent

n = 150
p = 43 / 150   # proportion observed

# works, but there's probably a better way
if h < p:
    right_tail = (stats.norm.cdf(h, p, np.sqrt(h * (1 - h) / n)))
    left_tail = (stats.norm.sf(h, h + (h - p), np.sqrt(h * (1 - h) / n)))
else:
    right_tail = (stats.norm.cdf(h, h + (h - p), np.sqrt(h * (1 - h) / n)))
    left_tail = (stats.norm.sf(h, p, np.sqrt(h * (1 - h) / n)))
    
print(right_tail, left_tail)
pval = right_tail + left_tail
print(pval)

0.36078986254455536 0.36078986254455536
0.7215797250891107


In [59]:
# unknown sigma

data = [1.11, 1.07, 1.11, 1.07, 1.12, 1.08, 0.98, 0.98, 1.02, 0.95, 0.95]
h = 1.00 # null hypothesis - mean < 1

n = len(data)
mu = np.mean(data)
sigma = np.std(data, ddof=1) # sample std so ddof = 1

pval = stats.t.cdf(h, n - 1, mu, sigma / np.sqrt(n)) 

print(pval)





0.03586064860607561


In [58]:
print(mu)
print(sigma)

1.0399999999999998
0.06587867636800247
